In [ ]:
import sys
# set CUDA_LAUNCH_BLOCKING=1
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
import numpy as np
from skimage.measure import label, regionprops
from skimage.transform import rotate
import matplotlib.pyplot as plt

# def extract_objects(mask):
#     """Extract individual objects from segmentation mask"""
#     labeled = label(mask)
#     return [mask[minr:maxr, minc:maxc] 
#             for region in regionprops(labeled) 
#             for (minr, minc, maxr, maxc) in [region.bbox]]

# def rotate_object(obj, angle):
#     """Rotate object by arbitrary angle with padding"""
#     return rotate(obj, angle, resize=True, order=0, 
#                  preserve_range=True).astype(bool)

# def random_placement(canvas, objects):
#     """Place objects randomly on empty canvas"""
#     h, w = canvas.shape
#     for obj in objects:
#         oh, ow = obj.shape
#         if (max_x := w - ow) >= 0 and (max_y := h - oh) >= 0:
#             x, y = np.random.randint(0, max_x+1), np.random.randint(0, max_y+1)
#             canvas[y:y+oh, x:x+ow] |= obj
#     return canvas

# # Example usage
# mask = np.zeros((128, 128), dtype=bool)
# mask[10:31, 10:31] = True  # Square object
# rr, cc = np.ogrid[:128, :128]
# mask[(rr - 80)**2 + (cc - 80)**2 <= 225] = True  # Circular object (radius 15)

# # Processing pipeline
# objects = extract_objects(mask)
# rotated = [rotate_object(obj, np.random.uniform(0, 360)) for obj in objects]
# result = random_placement(np.zeros_like(mask), rotated)

# # Visualization
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
# ax1.imshow(mask, cmap='gray'), ax1.set_title('Original Mask')
# # ax2.imshow(np.hstack([rotate_object(obj, 45) for obj in objects]),  # Example 45° rotation
# #            cmap='gray'), ax2.set_title('Rotated Objects')
# ax2.imshow(result, cmap='gray'), ax2.set_title('Final Composition')
# plt.show()


In [ ]:
import numpy as np
# import matplotlib.pyplot as plt
from cellpose import models, io
from cellpose.io import imread

io.logger_setup()

# model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
model = models.Cellpose(model_type='nuclei', gpu=True)

# list of files
# PUT PATH TO YOUR FILES HERE!
files = ['/group/jug/ashesh/data/sox2_one_img.tiff']

imgs = [imread(f) for f in files]
nimg = len(imgs)

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
channels = [[0,0]]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended)
# diameter can be a list or a single number for all images

masks, flows, styles, diams = model.eval(imgs, diameter=None, channels=channels)


### or to run one of the other models, or a custom model, specify a CellposeModel
# model = models.CellposeModel(model_type='livecell_cp3')

# masks, flows, styles = model.eval(imgs, diameter=30, channels=[0,0])

In [ ]:
import matplotlib.pyplot as plt
_,ax = plt.subplots(figsize=(10,5),ncols=2)
ax[0].imshow(masks[0])
ax[1].imshow(imgs[0])

In [ ]:
BACKGROUND_PATCHES= [ imgs[0][:200,:500].copy(), 
                     imgs[0][200:400,:400].copy(),
                     imgs[0][100:300,:400].copy(),
                     imgs[0][400:600,:300].copy()
                     ]

In [ ]:
from disentangle.data_loader.segmentation_augmentation import find_objects

In [ ]:
objects = find_objects(masks[0], imgs[0])

In [ ]:
_,ax = plt.subplots(figsize=(8,4),ncols=2)
idx = 5
ax[0].imshow(objects[idx][0])
ax[1].imshow(objects[idx][1])

## Rotate them.

In [ ]:
from disentangle.data_loader.segmentation_augmentation import get_background

In [ ]:
_,ax = plt.subplots(figsize=(18,3),ncols=6)
for i in range(6):
    ax[i].imshow(get_background((160, 160), BACKGROUND_PATCHES))

In [ ]:
from disentangle.data_loader.segmentation_augmentation import transform_object, render_objects
import torch

rendered_outputs = []
idx_list = np.random.randint(0, len(objects), 4)
for i in range(10):
    trans_objects = []
    for i in idx_list:
        tensor =torch.Tensor(objects[i]*1.0)
        new_obj = transform_object((tensor[0]*tensor[1])[None], max_angle=180)
        trans_objects.append(new_obj.squeeze())
        # print(obj.shape)

    output = render_objects(trans_objects, [0,1,2,3], [2], BACKGROUND_PATCHES)
    rendered_outputs.append(output)

In [ ]:
_,ax = plt.subplots(figsize=(15,6),ncols=5,nrows=2)
for i in range(10):
    ax[i//5][i%5].imshow(rendered_outputs[i])
    # ax[i//3][i%3].axis('off')

In [ ]:
from disentangle.data_loader.segmentation_augmentation import mask_to_boundary


In [ ]:
boundary = mask_to_boundary(objects[0][1], dilation_ratio=0.03)
_,ax = plt.subplots(figsize=(16,4),ncols=4)
ax[0].imshow(objects[0][0])
ax[1].imshow(objects[0][1])
ax[2].imshow(boundary)
ax[3].imshow(get_reduced_mask(objects[0][1], dilation_ratio=0.03))

In [ ]:
obj = objects[0][0]>0

In [ ]:
objects[0][1]